# Specifying activity models

*Activity models* are needed to accurately represent the *non-ideal thermodynamic behavior* of considered phases, i.e., when computing thermodynamic properties of phases and their species (e.g., species activities, chemical potentials, etc).

**Example**: for *an aqueous solution* at extreme saline conditions, an activity model designed for low salinity conditions will not perform well.

In Reaktoro, an **activity model assigned to a phase $\pi$** is a function for which the following inputs are given:

* temperature $T$ (in K),
* pressure $P$ (in Pa), and
* mole fractions of the species $x=(x_1,\ldots,x_\mathrm{N_{\pi}})$.

By default, all phases in Reaktoro are created with *ideal activity models* if not explicitly specified.

Consider the following chemical system:

In [ ]:
from reaktoro import *

# Define the PHREEQC database
db = PhreeqcDatabase("phreeqc.dat")

# Define phases
solution = AqueousPhase(speciate("H O Na Cl C Ca Mg"))
gases = GaseousPhase("CO2(g) H2O(g)")
solidsolution = MineralPhase("Siderite Rhodochrosite")
ionexchange = IonExchangePhase("NaX CaX2")

# Create the chemical system
system = ChemicalSystem(db, solution, gases, solidsolution, ionexchange)

It is equivalent to the one below:

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C Ca Mg"))
solution.setActivityModel(ActivityModelIdealAqueous())

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelIdealGas())

solidsolution = MineralPhase("Siderite Rhodochrosite")
solidsolution.setActivityModel(ActivityModelIdealSolution(StateOfMatter.Solid))

ionexchange = IonExchangePhase("NaX CaX2")
ionexchange.setActivityModel(ActivityModelIdealIonExchange())

system = ChemicalSystem(db, solution, gases, solidsolution, ionexchange)

*Non-ideal thermodynamic behavior* can differ from phase to phase for various conditions. Below, we elaborate on selecting and configuring different activity models for different phases.

## Non-ideal activity models for aqueous phases

The following are *non-ideal activity models* available for aqueous phases in Reaktoro:

* `ActivityModelDebyeHuckel`
* `ActivityModelDebyeHuckelKielland`
* `ActivityModelDebyeHuckelLimitingLaw`
* `ActivityModelDebyeHuckelParams`
* `ActivityModelDebyeHuckelPHREEQC`
* `ActivityModelDebyeHuckelWATEQ4F`
* `ActivityModelHKF`
* `ActivityModelPitzerHMW`

### Activity model for water and ionic species

The code below constructs a chemical system with an aqueous phase that uses the *HKF activity model* which is fairly accurate for solutions up to 6 molal NaCl.

In [ ]:
solution = AqueousPhase(speciate("H O Na Cl"))
solution.setActivityModel(ActivityModelHKF())

system = ChemicalSystem(db, solution)

Let's create a chemical state for this system (using [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) class), and set it so that we have a 1 molal NaCl saline solution. It contains chemical properties (provided by the class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html)) including the species' activity coefficients $\gamma_i$.

In [ ]:
state = ChemicalState(system)
state.temperature(25, "celsius")
state.pressure(1, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")

print(f"{'Species':<20}{'Activity Coefficient'}")
props = ChemicalProps(state)
for species in system.species():
    print(f"{species.name():<20}{props.speciesActivityCoefficient(species.name())}")

Note that the activity coefficients for neutral species other than H<sub>2</sub>O, i.e., `H2`, `NaOH`, and `O2`, are identical and equal to 1.25894. This is because most aqueous activity models are formulated for **the solvent water and ionic species** and **do not** properly compute activity coefficients for the neutral species.

### Correction for the neutral species

One way to improve this is to *combine HKF aqueous activity model* with *Setschenow model* (must be applied **for each** neutral species). Let us recreate the chemical system, in which a *chained activity model* is applied to the aqueous phase:

In [ ]:
solution = AqueousPhase(speciate("H O Na Cl"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelSetschenow("O2", 0.123),
    ActivityModelSetschenow("H2", 0.234),
    ActivityModelSetschenow("NaOH", 0.345),
))

system = ChemicalSystem(db, solution)

state = ChemicalState(system)
state.temperature(25, "celsius")
state.pressure(1, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")

print(f"{'Species':<20}{'Activity Coefficient'}")
props = ChemicalProps(state)
for species in system.species():
    print(f"{species.name():<20}{props.speciesActivityCoefficient(species.name())}")

Note that the activity coefficients of `O2`, `H2`, and `NaOH` are now different from each other. No changes were applied to the activity coefficients of `H2O` and ionic species.

### Activity models for aqueous CO<sub>2</sub>

CO<sub>2</sub> is a common gas dissolved in aqueous solutions. Most of the activity models above also do not properly calculate its activity (with exception of `ActivityModelPitzerHMW`). Thus, it is recommended to create a *chained activity model* with specific activity model for dissolved CO<sub>2</sub>, i.e.,

* `ActivityModelDrummond`
* `ActivityModelDuanSun`
* `ActivityModelRumpf`

**TASK 1**: 1) generate the system where the Drummond activity model is applied and 2) create 1-molal NaCl brine with 0.7 mol CO2 added to it 3) equilibrate and evaluate CO2(aq) activity coefficient

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

## Non-ideal activity models for gaseous and liquid phases

Reaktoro implements a general form of the *cubic equation of state* from which classic models are derived. You can assign one of the following models to an [GaseousPhase](https://reaktoro.org/api/classReaktoro_1_1GaseousPhase.html) or [LiquidPhase](https://reaktoro.org/api/classReaktoro_1_1LiquidPhase.html) object:

* `ActivityModelPengRobinson`
* `ActivityModelRedlichKwong`
* `ActivityModelSoaveRedlichKwong`
* `ActivityModelVanDerWaals`

Additionally, Reaktoro implements some equations of state designed for specific gaseous phases:

* `ActivityModelSpycherPruessEnnis` (for H<sub>2</sub>O-CO<sub>2</sub> gas mixtures)
* `ActivityModelSpycherReed` (for H<sub>2</sub>O-CO<sub>2</sub>-CH<sub>4</sub> gas mixtures)

The example below demonstrates the use of the Peng-Robinson equation of state as the activity model for a gaseous phase:

In [ ]:
gases = GaseousPhase("CO2(g) CH4(g) H2O(g) O2(g) H2(g)")
gases.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, gases)

Let's create a chemical state for this system and evaluate the fugacity coefficients of the gases (included in thermodynamic and chemical properties):

In [ ]:
state = ChemicalState(system)
state.temperature(100.0, "celsius")
state.pressure(1.0, "bar")
state.set("CO2(g)", 0.80, "mol")
state.set("CH4(g)", 0.10, "mol")
state.set("H2O(g)", 0.05, "mol")
state.set("O2(g)",  0.03, "mol")
state.set("H2(g)",  0.02, "mol")

props = ChemicalProps(state)

print(f"{'Gas':<10}{'Fugacity Coefficient'}")
for i in range(system.species().size()):
    print(f"{system.species(i).name():<10}{props.speciesActivityCoefficient(i)}")

The full list of properties of the system can be inspected using function `print`:

In [ ]:
print(props)

## Activity models for pure mineral and condensed phases

*Pure minerals and condensed phases* (substances in solid or liquid states) are normally represented with a single species. Their chemical behavior is governed by ideal activity model.

## Activity models for solid solution phases

Currently, Reaktoro supports only the following activity models for solid solutions:

* `ActivityModelRedlichKister`

Below, we create a chemical system with a single solid solution phase (formed with minerals K-feldspar and albite) governed by the Redlich-Kister activity model, creates the chemical state based on this system, and prints its chemical properties:

In [ ]:
a0, a1, a2 = 1.0, 2.0, 3.0  # the Redlich-Kister parameters (justdemonstration values!)

solidsolution = SolidPhase("K-feldspar Albite")
solidsolution.setActivityModel(ActivityModelRedlichKister(a0, a1, a2))

system = ChemicalSystem(db, solidsolution)

state = ChemicalState(system)
state.set("K-feldspar", 0.5, "mol")
state.set("Albite", 0.5, "mol")

props = ChemicalProps(state)
print(props)

## Activity models for ion exchange phases

For non-ideal activity models for ion exchange phases, the following is currently available in Reaktoro:

* `ActivityModelIonExchangeGainesThomas`
* `ActivityModelIonExchangeVanselow`

Below, we create a chemical system with aqueous and ion exchange phases and corresponding chemical state as well compute and output system's chemical properties:

In [ ]:
db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase("H2O Na+ Cl- H+ OH- K+ Ca+2 Mg+2")
solution.setActivityModel(ActivityModelDebyeHuckel())

exchange = IonExchangePhase("NaX KX CaX2 MgX2")
exchange.setActivityModel(ActivityModelIonExchangeGainesThomas())

system = ChemicalSystem(db, solution, exchange)

state = ChemicalState(system)
state.set("H2O" , 1.00, "kg")
state.set("Na+" , 1.00, "mmol")
state.set("K+"  , 1.00, "mmol")
state.set("Mg+2", 1.00, "mmol")
state.set("Ca+2", 1.00, "mmol")
state.set("NaX" , 0.06, "umol")
state.set("KX" ,  0.02, "umol")
state.set("CaX2" ,0.01, "umol")
state.set("MgX2" ,0.01, "umol")

props = ChemicalProps(state)
print(props)

**TASK 2**: equilibrate the ion-exchange problem above and investigate which things are calculated when evaluating ion-exchange properties.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

**Note**: The correct selection and attachment of activity model to a phase is vital. *Special care must be applied* when comparing Reaktoro's computations with other codes (up to considering the identical set of parameters for the activity model).